## Preprocessing data

## Libraries

In [1]:
import pandas as pd

## Processing Generation Dataset

### Load dataset

In [2]:
# Load generation dataset
df_gen = pd.read_excel('../data/in/endog/generation_2020_2025.xlsx')
df_gen.head()

,Fecha,Recurso,Tipo Generación,Combustible,Tipo Despacho,Es Menor,Clasificación,0,1,2,...,15,16,17,18,19,20,21,22,23,Version
0,2020-01-01,AGPE - ECOPETROL LA HORMIGA,TERMICA,GAS,NO DESPACHADO CENTRALMENTE,NO,AUTOG PEQ. ESCALA,226.51,223.84,223.12,...,269.53,273.34,273.69,227.1,225.26,256.94,155.93,10.91,11.59,TX5
1,2020-01-01,AGUA FRESCA,HIDRAULICA,AGUA,NO DESPACHADO CENTRALMENTE,SI,NORMAL,7268.40,7268.40,7270.80,...,7024.80,6937.20,6874.80,6802.8,6764.40,6729.60,6714.00,5462.40,5950.80,TX5
2,2020-01-01,ALBAN,HIDRAULICA,AGUA,DESPACHADO CENTRALMENTE,NO,NORMAL,251520.00,251040.00,251280.00,...,238880.00,238880.00,238960.00,244786.0,246944.00,248476.00,250296.00,250590.00,250340.00,TX5
3,2020-01-01,ALEJANDRÍA,HIDRAULICA,AGUA,NO DESPACHADO CENTRALMENTE,SI,NORMAL,14861.00,14608.00,14322.00,...,8536.00,8338.00,8525.00,10065.0,12892.00,14839.00,14927.00,14982.00,14872.00,TX5
4,2020-01-01,ALTO TULUA,HIDRAULICA,AGUA,NO DESPACHADO CENTRALMENTE,SI,NORMAL,0.00,0.00,0.00,...,10908.00,10908.00,10911.60,10908.0,10908.00,10915.20,10915.20,10918.80,10922.40,TX5


### Validate missing values

In [3]:
print("Are there missing values in the dataset?: ", df_gen.isnull().values.any())
print("Missing values by column:")
df_gen.isnull().sum()

Are there missing values in the dataset?:  True
Missing values by column:


Fecha               0
Recurso             0
Tipo Generación     0
Combustible         0
Tipo Despacho       0
Es Menor            0
Clasificación       0
0                  36
1                   5
2                   6
3                   2
4                   0
5                   2
6                   4
7                   5
8                   3
9                   8
10                  2
11                  2
12                  6
13                  3
14                  6
15                  2
16                  0
17                  1
18                  3
19                 50
20                  1
21                  2
22                  3
23                  6
Version             0
dtype: int64

In this case, the missing values represent a contribition of zero (0) to energy generation. Therefore, the missing values will be replaced with 0.

In [4]:
df_gen = df_gen.fillna(0)

In [5]:
print("Are there missing values in the dataset? (after replacement): ", df_gen.isnull().values.any())
print("Missing values by column (after replacement):")
df_gen.isnull().sum()

Are there missing values in the dataset? (after replacement):  False
Missing values by column (after replacement):


Fecha              0
Recurso            0
Tipo Generación    0
Combustible        0
Tipo Despacho      0
Es Menor           0
Clasificación      0
0                  0
1                  0
2                  0
3                  0
4                  0
5                  0
6                  0
7                  0
8                  0
9                  0
10                 0
11                 0
12                 0
13                 0
14                 0
15                 0
16                 0
17                 0
18                 0
19                 0
20                 0
21                 0
22                 0
23                 0
Version            0
dtype: int64

### Standardize "Tipo Generación" values

In [6]:
df_gen['Tipo Generación'].unique()

array(['TERMICA', 'HIDRAULICA', 'SOLAR', 'COGENERADOR', 'EOLICA', 'Solar'],
      dtype=object)

Convert all values ​​in the 'Tipo Generación' column to uppercase

In [7]:
df_gen['Tipo Generación'] = df_gen['Tipo Generación'].str.upper()
df_gen['Tipo Generación'].unique()

array(['TERMICA', 'HIDRAULICA', 'SOLAR', 'COGENERADOR', 'EOLICA'],
      dtype=object)

After validating that there are not missing values in the dataset, the next step is to transform the dataset into the appropiate format.

### Melt columns hours (0-23) -> wide to long format

In [8]:
# Melt columns hours (0-23) -> wide to long format
df_gen_long = df_gen.melt(
    id_vars = ["Fecha", "Tipo Generación"], # Fixed columns
    value_vars = [str(i) for i in range(24)], # Columns to melt (0 to 23)
    var_name = "Hora", # New column name to hours
    value_name="generacion_kwh" # New column name for generation values
)

df_gen_long.head(10)

,Fecha,Tipo Generación,Hora,generacion_kwh
0,2020-01-01,TERMICA,0,226.51
1,2020-01-01,HIDRAULICA,0,7268.40
2,2020-01-01,HIDRAULICA,0,251520.00
3,2020-01-01,HIDRAULICA,0,14861.00
4,2020-01-01,HIDRAULICA,0,0.00
5,2020-01-01,HIDRAULICA,0,13917.60
6,2020-01-01,HIDRAULICA,0,625.96
7,2020-01-01,HIDRAULICA,0,72659.88
8,2020-01-01,HIDRAULICA,0,482.71
9,2020-01-01,HIDRAULICA,0,14442.60


### Group and Pivot
Group by Date, Hour and Generation Type (summarized)

In [9]:
# Group by Fecha, Hora and Tipo Generación (summarized)

df_gen_pivot = df_gen_long.groupby(["Fecha", "Hora", "Tipo Generación"])["generacion_kwh"].sum().unstack()
df_gen_pivot.head(25)

Tipo Generación           COGENERADOR    EOLICA  HIDRAULICA      SOLAR  \
Fecha               Hora                                                 
2021-01-01 00:00:00 0        21522.11   3328.10  5244418.15       0.00   
                    1        21766.38   4063.22  5117544.23       0.00   
                    10       21301.39  10220.62  5118684.25  102386.27   
                    11       22571.79   8472.44  5285404.70  110299.13   
                    12       22660.40   9043.93  5382742.33  102263.57   
                    13       22550.31   8886.60  5387610.12   89491.00   
                    14       22865.93   9317.35  5323733.70   79919.87   
                    15       22454.38  10573.28  5257803.84   64496.33   
                    16       22528.86   9816.01  5129654.92   30274.20   
                    17       18137.58   9010.70  5059082.54    4330.74   
                    18       22184.16   7904.03  5930842.16       2.83   
                    19       22143.85   5922.55  6113893.76       0.00   
                    2        20976.76   4139.08  4919389.58       0.00   
                    20       23381.23   3272.15  5962927.11       0.00   
                    21       22711.83   3660.03  5834559.54       0.00   
                    22       21119.85   5925.16  5505832.43       0.00   
                    23       21000.89   6858.90  5255505.82       0.00   
                    3        21212.18   2114.45  4702267.19       0.00   
                    4        19867.94   2070.41  4759095.19       0.00   
                    5        20321.55   2669.70  4808062.23       0.00   
                    6        20821.96   2117.35  4465756.48    3941.44   
                    7        21389.87   3748.52  4512878.18   27233.95   
                    8        21362.03   7018.91  4747889.55   53292.43   
                    9        21418.93   9077.05  4962393.02   76981.63   
2021-01-02 00:00:00 0        20016.50   7592.47  5003290.22      33.18   

Tipo Generación              TERMICA  
Fecha               Hora              
2021-01-01 00:00:00 0     1391375.95  
                    1     1325940.54  
                    10    1017685.76  
                    11    1044030.63  
                    12    1096915.81  
                    13    1161082.10  
                    14    1180372.44  
                    15    1160653.89  
                    16    1280373.91  
                    17    1435643.03  
                    18    1458504.16  
                    19    1515753.50  
                    2     1291073.68  
                    20    1530709.05  
                    21    1448846.19  
                    22    1429770.14  
                    23    1381787.17  
                    3     1326887.72  
                    4     1149583.09  
                    5      997739.48  
                    6      945300.31  
                    7      906850.42  
                    8      931573.52  
                    9      982809.61  
2021-01-02 00:00:00 0     1315291.41

## Processing Spot Prices Dataset

### Load dataset

In [10]:
# Load dataset
df_price = pd.read_excel('../data/in/endog/spot_prices_2020_2025.xlsx')
df_price.head()

,Fecha,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,Versión
0,2020-01-01,72.02,136.71,127.71,127.71,127.71,72.02,72.02,72.02,72.02,...,136.67,127.71,72.02,183.93,406.73,213.73,213.73,213.73,136.71,TX5
1,2020-01-02,71.34,71.34,71.34,71.34,71.34,73.95,73.95,137.88,185.10,...,357.80,357.80,307.90,397.90,387.90,357.80,307.90,307.90,307.90,TX5
2,2020-01-03,116.31,116.31,126.29,126.29,135.29,126.29,123.46,275.31,275.31,...,275.31,275.31,275.31,285.31,285.31,275.31,275.31,275.31,275.31,TX5
3,2020-01-04,144.36,144.36,144.36,144.36,134.34,144.36,134.34,144.36,188.36,...,255.36,255.36,255.36,255.36,255.36,255.36,255.36,255.36,188.36,TX5
4,2020-01-05,209.12,209.12,188.12,188.12,188.12,188.12,180.62,157.12,188.12,...,209.12,188.12,209.12,305.12,305.12,305.12,305.12,305.12,209.12,TX5


### Validate missing values

In [11]:
print("Are there missing values in the dataset?: ", df_price.isnull().values.any())
print("Missing values by column:")
df_price.isnull().sum()

Are there missing values in the dataset?:  False
Missing values by column:


Fecha      0
0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
Versión    0
dtype: int64

There are not missing values in the dataset. Therefore, no processing is necessary.

### Melt columns hours (0-23) -> wide to long format

In [12]:
# Derretir columnas de horas (0-23)
df_price_long = df_price.melt(
    id_vars = ["Fecha"], # Fixed columns
    value_vars = [str(i) for i in range(24)], # Columns to melt (0 to 23)
    var_name = "Hora", # New column name to hours
    value_name = "Precio" # New column name for price values
)

df_price_long.head()

,Fecha,Hora,Precio
0,2020-01-01,0,72.02
1,2020-01-02,0,71.34
2,2020-01-03,0,116.31
3,2020-01-04,0,144.36
4,2020-01-05,0,209.12


## Processing ENSO dataset

### Load dataset

In [13]:
# Load dataset
df_enso = pd.read_csv("../data/in/exog/ENSO_2020_2025.csv", parse_dates=["FECHA"])
df_enso.head()

,FECHA,ENSO,NIVEL_ENSO
0,2020-01-01,0.5,1
1,2020-01-02,0.5,1
2,2020-01-03,0.5,1
3,2020-01-04,0.5,1
4,2020-01-05,0.5,1


### Validate missing values

In [14]:
print("Are there missing values in the dataset?: ", df_enso.isnull().values.any())
print("Missing values by column:")
df_enso.isnull().sum()

Are there missing values in the dataset?:  False
Missing values by column:


FECHA         0
ENSO          0
NIVEL_ENSO    0
dtype: int64

### Drop columns

In [15]:
df_enso = df_enso.drop(columns=["ENSO"])
df_enso.head()

,FECHA,NIVEL_ENSO
0,2020-01-01,1
1,2020-01-02,1
2,2020-01-03,1
3,2020-01-04,1
4,2020-01-05,1


There are not missing values in the dataset. Therefore, no processing is necessary.

## Merge datasets

### Merge generation and spot prices datasets

In [16]:
# Merge by Fecha & Hour
df = pd.merge(
    df_gen_pivot,
    df_price_long,
    on = ["Fecha", "Hora"],
    how = "inner" # Keep only existing dates-hours presents in both datasets
)

df.head()

,Fecha,Hora,COGENERADOR,EOLICA,HIDRAULICA,SOLAR,TERMICA,Precio
0,2021-01-01 00:00:00,0,21522.11,3328.10,5244418.15,0.00,1391375.95,142.31
1,2021-01-01 00:00:00,1,21766.38,4063.22,5117544.23,0.00,1325940.54,142.31
2,2021-01-01 00:00:00,10,21301.39,10220.62,5118684.25,102386.27,1017685.76,142.31
3,2021-01-01 00:00:00,11,22571.79,8472.44,5285404.70,110299.13,1044030.63,142.31
4,2021-01-01 00:00:00,12,22660.40,9043.93,5382742.33,102263.57,1096915.81,153.31


### Order columns

In [17]:
# Order columns (Fecha, Hora, Precio, Tipos de Generación)
column_order = ["Fecha", "Hora", "Precio"] + df_gen["Tipo Generación"].unique().tolist()

# Reeplace NaN with 0 value
df = df[column_order].fillna(0)

# Correct columns names (UPPER CASE) (eg.: "Solar" vs "SOLAR")
df.columns = [col.upper() for col in df.columns]

df.head()

,FECHA,HORA,PRECIO,TERMICA,HIDRAULICA,SOLAR,COGENERADOR,EOLICA
0,2021-01-01 00:00:00,0,142.31,1391375.95,5244418.15,0.00,21522.11,3328.10
1,2021-01-01 00:00:00,1,142.31,1325940.54,5117544.23,0.00,21766.38,4063.22
2,2021-01-01 00:00:00,10,142.31,1017685.76,5118684.25,102386.27,21301.39,10220.62
3,2021-01-01 00:00:00,11,142.31,1044030.63,5285404.70,110299.13,22571.79,8472.44
4,2021-01-01 00:00:00,12,153.31,1096915.81,5382742.33,102263.57,22660.40,9043.93


In [18]:
# Order DataFrame by Date and Hour

# FECHA: string to date
df["FECHA"] = pd.to_datetime(df["FECHA"])

# Hora: string to int
df["HORA"] = df["HORA"].astype(int)

# Order by Fecha y Hora
df = df.sort_values(
    by=["FECHA", "HORA"], 
    ascending=[True, True]
).reset_index(drop=True)  # Reset index to maintain order

df.head(25)

,FECHA,HORA,PRECIO,TERMICA,HIDRAULICA,SOLAR,COGENERADOR,EOLICA
0,2020-01-01,0,72.02,2109637.93,4418103.51,0.00,28880.74,4291.34
1,2020-01-01,1,136.71,1864989.80,4548853.97,0.00,29107.37,3511.94
2,2020-01-01,2,127.71,1720883.01,4525898.75,0.00,25939.87,2641.90
3,2020-01-01,3,127.71,1649014.61,4458645.56,0.00,25651.57,3032.30
4,2020-01-01,4,127.71,1711745.36,4279657.90,0.00,25245.37,2514.76
5,2020-01-01,5,72.02,1571431.28,4311552.05,0.00,22696.46,980.62
6,2020-01-01,6,72.02,1297795.61,4173921.34,2330.51,19751.12,457.47
7,2020-01-01,7,72.02,1257262.31,4231331.52,23088.00,18429.91,0.00
8,2020-01-01,8,72.02,1339329.04,4334616.11,43158.98,19297.62,526.95
9,2020-01-01,9,72.02,1486541.89,4437534.05,49153.18,19783.46,1515.43


### Merge ENSO dataset

In [19]:
# Merge by FECHA

df = df.merge(
    df_enso,
    on="FECHA",
    how="left"
)

df.head(10)

,FECHA,HORA,PRECIO,TERMICA,HIDRAULICA,SOLAR,COGENERADOR,EOLICA,NIVEL_ENSO
0,2020-01-01,0,72.02,2109637.93,4418103.51,0.00,28880.74,4291.34,1.0
1,2020-01-01,1,136.71,1864989.80,4548853.97,0.00,29107.37,3511.94,1.0
2,2020-01-01,2,127.71,1720883.01,4525898.75,0.00,25939.87,2641.90,1.0
3,2020-01-01,3,127.71,1649014.61,4458645.56,0.00,25651.57,3032.30,1.0
4,2020-01-01,4,127.71,1711745.36,4279657.90,0.00,25245.37,2514.76,1.0
5,2020-01-01,5,72.02,1571431.28,4311552.05,0.00,22696.46,980.62,1.0
6,2020-01-01,6,72.02,1297795.61,4173921.34,2330.51,19751.12,457.47,1.0
7,2020-01-01,7,72.02,1257262.31,4231331.52,23088.00,18429.91,0.00,1.0
8,2020-01-01,8,72.02,1339329.04,4334616.11,43158.98,19297.62,526.95,1.0
9,2020-01-01,9,72.02,1486541.89,4437534.05,49153.18,19783.46,1515.43,1.0


In [20]:
# Add column FECHA_HORA (eg.: 2020-01-01 01:00:00)
df["FECHA_HORA"] = pd.to_datetime(df["FECHA"]) + pd.to_timedelta(df["HORA"], unit="h")


# Reorder columns 
column_order = ["FECHA_HORA"] + [col for col in df.columns if col != "FECHA_HORA"]
df = df[column_order]

df.head()

,FECHA_HORA,FECHA,HORA,PRECIO,TERMICA,HIDRAULICA,SOLAR,COGENERADOR,EOLICA,NIVEL_ENSO
0,2020-01-01 00:00:00,2020-01-01,0,72.02,2109637.93,4418103.51,0.0,28880.74,4291.34,1.0
1,2020-01-01 01:00:00,2020-01-01,1,136.71,1864989.80,4548853.97,0.0,29107.37,3511.94,1.0
2,2020-01-01 02:00:00,2020-01-01,2,127.71,1720883.01,4525898.75,0.0,25939.87,2641.90,1.0
3,2020-01-01 03:00:00,2020-01-01,3,127.71,1649014.61,4458645.56,0.0,25651.57,3032.30,1.0
4,2020-01-01 04:00:00,2020-01-01,4,127.71,1711745.36,4279657.90,0.0,25245.37,2514.76,1.0


## Export final Dataset

In [21]:
#df["FECHA"] = df["FECHA"].dt.strftime('%Y-%m-%d')
#df.to_csv('../data/out/dataset_gen_spot_exog.csv', index=False)

print("Final Dataset exported...")

Final Dataset exported...
